## PrettyTables.jl

Dieses Paket erlaubt es uns, Typen wie Arrays oder Dicts in gut lesbarer Tabellenform auszugeben:

In [ ]:
using Pkg; Pkg.add("PrettyTables")
using PrettyTables

In [ ]:
M = rand(4,4)
pretty_table(M)

Die Ausgabe kann nicht nur in der Konsole, sondern auch in HTML oder LaTeX erfolgen (praktisch für Abschlussarbeiten):

In [ ]:
dict = Dict(1 => "Jan", 2 => "Feb", 3 => "Mar", 4 => "Apr", 5 => "May", 6 => "Jun");
pretty_table(dict)

In [ ]:
@ptconf(
	backend = Val(:latex),
	highlighters = LatexHighlighter((data,i,j) -> data[i,j] > 0.5, "textbf"),
	wrap_table = false,
	alignment = :c
)

In [ ]:
@ptconfclean

In [ ]:
@pt :header = ["Player $i" for i = 1:4] M

In [ ]:
open("tablefile.txt", "w") do io
    pretty_table(io, M; header = ["Player $i" for i = 1:4])
end

## String-Manipulation

Wir erstellen sogenannte regular expressions (regex) via *non-standard string literals*, die mit `r` beginnen:

In [ ]:
re = r"^\s*(?:#|$)"
typeof(re)

Wir nutzen regex, um ein bestimmtes Zeichenpattern zu kodieren. Beispielsweise können wir testen, ob E-Mail-Adressen in einem String vorkommen:

In [ ]:
email_pattern = r".+@.+"
input = "john.doe@mit.edu"
println(occursin(email_pattern, input)) #> true

Dieser Regex-String erfasst Kreditkartennummern, die mit der Ziffer 4 beginnen und entweder 16 Ziffern oder 13 Ziffern haben (wenn die Nummer einen 3-stelligen Sicherheitscode enthält):

In [ ]:
visa = r"^(?:4[0-9]{12}(?:[0-9]{3})?)$"
input = "4457418557635128"
occursin(visa, input)

In [ ]:
if occursin(visa, input)
    println("credit card found")
    m = match(visa, input)
    println(m.match)
    println(m.offset)
    println(m.offsets)
end

In [ ]:
visa = r"(?:4[0-9]{12}(?:[0-9]{3})?)$"
input = "foo4457418557635128"
occursin(visa, input)

- `^` Matcht den Anfang des Strings
- `(?:...)` erstellt eine non-capturing group
- `4` erfasst die Zahl 4
- `[0-9]{12}` erfasst 12 Ziffern in der range 0-9
- `(?:[0-9]{3})?` matcht 3 Ziffern in der range 0-9 optional (i.e. the security code)
- `?` macht die non-capturing group optional
- `$` Matcht das Ende des Strings

Ein anderes Beispiel wäre `\w`, das Wortzeichen (Buchstaben, Zahlen oder Unterstriche) erfasst.

In [ ]:
occursin(r"\w the pattern", "string containing the pattern")

In [ ]:
occursin(r"\w the pattern", "string containin~ the pattern")

Wichtige Befehle sind:
- `occursin`
- `match`
- `replace`
- `repeat`
- `join`
- `split`
- `match`

Zusaätzlich zur Julia-Dokumentation gibt es praktische Tools wie [RegExr](https://regexr.com/), die uns das Arbeiten mit regex erleichtern.

## Zurück zur Datenanalyse

Wir betrachten wieder den gleichen Datensatz wie in Termin 1 und laden zunächst alle dafür benötigten packages rein sowie alle ändereungen am DataFrame die wir bereits vorgenommen hatten.

In [ ]:
using DataFrames
using XLSX
using Dates 
using Plots 
using Pipe 
using Statistics

In [ ]:
file_path = "./CMO-Historical-Data-Monthly.xlsx"
sheet_name = "Monthly Prices"

# Read the Excel file
xlsx_data = XLSX.readdata(file_path, sheet_name, "A5:BT754")

col_names = xlsx_data[1, :]
df = DataFrame(xlsx_data[4:end, :], Symbol.(col_names))

rename!(df, 1 => "date")

function transform_missing(df)
    for col in names(df)
        df[!, col] = ifelse.((df[!, col] .== "…") .| (df[!, col] .== "..") , missing, df[!, col])
    end
end

function transform_dates(date)
    date = String.(date)
    new_date = replace(date,"M" => "-")
    return Date(new_date,dateformat"yyyy-mm")
end 

transform!(df, :date  => ByRow(transform_dates),renamecols = false)

transform_missing(df)

first(df,10)

### Aufgabe 1
Ersetze in jeder Spalte Kommata und Leerzeichen durch Unterstriche

## Wide vs. Long Format

Datensätze werden häufig in zwei Formaten dargestellt long und wide. Bis jetzt haben wir uns nur das wide format angeschaut, in dem die Daten in Matrixform, dh das Beobachtungen über mehrere Variablen in einer Zeile dargestellt werden. Im long format wird ändert sich die zu einer Beobachtung über eine Variable.
um von wide zu long zu gelagen benutzen wir die ``stack`` Funktion (vice versa ``unstack``)

In [ ]:
df_long = stack(df)
rename!(df_long, 2 => "name")

#unstack(df_long,:name,:value) 

Unterschied zwischen wide und long format in der Praxis:

In [ ]:
# create some example data in a wide format
df_example = DataFrame(x=[1, 2, 3], y1=[2, 3, 4], y2=[5, 6, 7])

In [ ]:
scatter(df_example.x, df_example.y1, markersize=10, label="Group 1")
scatter!(df_example.x, df_example.y2, markersize=10, label="Group 2")
# ...

In [ ]:
# same data in long format
x = repeat(1:3, inner=2)
y = [2, 5, 3, 6, 4, 7]
group = repeat([1, 2], outer=3)
df_example = DataFrame(:x => x, :y => y, :group => group)

In [ ]:
scatter(df_example.x, df_example.y, group=df_example.group, markersize=10, label=["Group 1" "Group 2"])

## Advanced plotting

Das package Plots package in Julia kann auf verschiedene backends zurückgreifen, welche die Optik als auch einige Funktionalitäten
verändert. Eines welches für die Datenanalyse nütlich sein kann ist ``plotlyjs``. 

In [ ]:
plotlyjs()

Um zum ursprünglichen Backend zurückzukommen kann dieses analog mit ``gr()`` wieder aufgerufen werden.

In [ ]:
gr()
x = df_long.date
y = df_long.value

years = [i for i in range(df_long.date[1],df_long.date[end],step = Year(5))]
year_label = ["$year" for year in range(year(df_long.date[1]),year(df_long.date[end]),step = 5)]

plot(x,y,
    group = df_long.name,
    legend = :outertopright,
    xticks = (years,year_label),
    xrotation = 90,
    visible = "legendonly"
)

 In der explorativen Datenanalyse werden häufig Korrelationsplots verwendet, um eine Übersicht zu den Abhängigkeitsstrukturen der verschiedenen Merkmale zu erlangen. Hierfür importieren wir das StatsBase package um auf die Funktion ``pairwise`` zurückgreifen zu können, 
 welche es uns ermöglicht Spalten paarweise auszuwählen und die Korrelationen zu berechnen.

In [ ]:
using Pkg; Pkg.add("StatsBase")

In [ ]:
using StatsBase
df_cor = @pipe df |>  select(_, r"Crude_oil","Shrimps_Mexican", "Groundnuts")
pairwise(df_cor, eachcol(_), skipmissing = :pairwise)

In [ ]:
gr(size = (600,600))

colnames = names(select(df_cor, r"Crude_oil", "Shrimps_Mexican", "Groundnuts"))

heatmap(cor(Matrix(df_cor)),
    xticks = (1:6,colnames),
    yticks = (1:6,colnames),
    xrotation = 90,
    color= cgrad([:white,:dodgerblue4]),
    annotate = ([(j, i, text(round(df_cor[i,j],digits=3),8)) for i in 1:6 for j in 1:6]),
)

Neben dem ``Plots`` package gibt es viele weitere plotting packages, die je nach Präferenz und Anwendungsfall sinnvoll sein könnten.
Eines davon welches sich für die statistische Datenanalyse gut eignet ist das ``StatsPlot`` package und stellt eine Erweiterung zu ``Plots`` dar. [Hier](https://docs.juliaplots.org/dev/generated/statsplots/) findet ihr eine genauere Dokumentation. 


In [ ]:
using StatsPlots
plotlyjs(size = (500,500))

@df df corrplot([:Gold :Maize :Beef :Orange], 
    grid = false,
    markercolor = :auto)

Auch lineare Modelle können in Julia mit geringem Aufwand gelöst werden.

In [ ]:
using GLM

fm2 = @formula(Crude_oil_Brent ~ Crude_oil_WTI)

#fm = @formula(Silver ~ Aluminum)
model = lm(fm2, df)

Die Funktion berechnet hierbei den kq-Schätzer für $\beta$ in dem Modell

$Y = X \beta + \epsilon$,

mit $\epsilon$ ~ iid.

### Aufgabe 2
Erstellt einen interaktiven Korellationsplot über alle Spalten des DataFrames. 

In der Zeitreihenanalyse werden häufig sogenannte autoregressive Modelle (AR) benutzt. Dabei ist das AR(p) Modell für $ p \in \N $
 definiert als

 $a_t = \sum_{i=1}^p c_i a_{t-1} + \epsilon$
 
für (meist) unbekannte Konstanten $c_1,...,c_p \in \R$. Schätzt zunächst für den Preisprozess der Rohölsorte WTI ein AR(1)-Modell mittels der ``lm`` Funktion.

Erweitert dann das Modell, indem ihr einen weiteren Zeitschritt der Brent Marke mit in die Schätzung aufnehmt. Kann dadurch die erklärte Varianz des Modells (angepasstes R
) verbessert werden? Hinweis: r2()

## Shapefiles (Geostatistik)

In [ ]:
using Pkg; Pkg.add("Shapefile")

In [ ]:
using Shapefile, Plots

# Read the Shapefile into a DataFrame
shapes = Shapefile.Table("NUTS_RG_60M_2021_3035.shp")

In [ ]:
shapes |> DataFrame

In [ ]:
# Create a plot of the polygons
geoms = Shapefile.shapes(shapes)
plotlyjs()
plot(geoms, aspect_ratio=1)

In [ ]:
shapes.CNTR_CODE

In [ ]:
# die Funktion filter gibt es leider nicht, daher eigene Funktion
function selectshapes(table)
    geoms = empty(Shapefile.shapes(table))
    for row in table
        if !ismissing(row.CNTR_CODE) && row.CNTR_CODE == "DE"
            push!(geoms, Shapefile.shape(row))
        end
    end
    return geoms
end

@pipe shapes |> 
    selectshapes(_) |>
    plot(_, aspect_ratio=1)